#  NITF Replacement Sensor Model (RSM) Demo

This notebook demonstrates how to create an RSM TRE for a given image.  

## Benefits of the RSM over "4-corners" and SENSRB

1. No knowledge of the underlying camera is required.  The embedded polynomials abstract everything.
2. The imagery provider (You), no longer need to actually ortho-rectify the imagery.  Simply convert the imagery to the expected color-space, apply any color corrections, then write to disk.  The RSM can allow a downstream user to optionally apply ortho-rectification.

## Step 0 - Setup Preconditions

First, we have our required Python packages.

In [ ]:
import datetime
import os
import sys

import pandas as pd

import cv2

import numpy as np

import plotly.graph_objects as go
import plotly.express       as px
import plotly.subplots      as sp

sys.path.append( '../src' )
import tmns.net.wms as wms

Our image is from April 23, 2002.

In [ ]:
img_path = 'ARUDEN000040045.tif'

We will be verifying our Ground Control Points via a comparison against USDA NAIP imagery.  This is available from the US Government via a Web-Map-Service (WMS).

In [ ]:
#  URL to ArcGIS Hosted NAIP imagery
wms_url = 'https://gis.apfo.usda.gov/arcgis/services/NAIP/USDA_CONUS_PRIME/ImageServer/WMSServer?service=WMS'

# USDA_CONUS_PRIME
wms_layers = 0

#  Image Format
wms_format = 'image/png'

## Step 1 - Load Non-Orthorectified Image
In this demo, we have an aerial photograph of Denver's City Park taken in 2002.  This photo is used because it is not orthorectified.  We will need to manually determine a camera model.

In [ ]:
img_bands = cv2.cvtColor( cv2.imread( img_path, cv2.IMREAD_COLOR ), cv2.COLOR_BGR2RGB )
print(img_bands.shape)

In [ ]:
fig1 = go.Figure()
fig1.add_trace( go.Image( z = img_bands ) )
fig1.update_layout( title = 'USGS Aerial Photo, 4/23/2002',
                    height = 900 )
fig1.show( renderer = 'png' )

## Estimating Rigid Sensor Model

For this exerciese, we use the available metadata from the USGS for the given frame. 

In [ ]:
img_df = pd.DataFrame( { 'Attribute': ['Acquisition Date', 'Flight Altitude (ft)', 'Focal Length (mm)',
                                       'Film Width (mm)', 'Film Height (mm)',
                                       'Center Lat', 'Center Lon',
                                       'NW Lat', 'NW Lon', 'NE Lat', 'NE Lon',
                                       'SE Lat', 'SE Lon', 'SW Lat', 'SE Lon',
                                       'Pitch Angle', 'Yaw Angle', 'Roll Angle' ],
                         'Value': [ datetime.datetime( year = 2002, month = 4, day = 23 ),
                                    7500, 152.77, 229, 229,
                                    39.750006, -104.95283,
                                    39.765563, -104.972699, 39.765335, -104.932675,
                                    39.734448, -104.932979, 39.734676, -104.972985,
                                    -1.3, -8.7, 0.0 ] } )
display( img_df )

We will assume the aircraft position is directly above the center point.  Given the camera rotation is not perfectly level, this will require adjustments later. 

In [ ]:
position_lla = [ img_df.loc[img_df['Attribute'] == 'Center Lon'].values[0][1],
                 img_df.loc[img_df['Attribute'] == 'Center Lat'].values[0][1],
                 img_df.loc[img_df['Attribute'] == 'Flight Altitude (ft)'].values[0][1] * 3.28084]
display( position_lla )

## Mark Ground Control Points

We have a set of Ground-Control-Points in the attached file. The first few are shown for context.

In [ ]:
gcp_df = pd.read_csv( 'GCPs.csv' )
display( gcp_df.head(5) )
display( f'Total of {gcp_df.shape[0]} GCPs loaded' )

### Verifying Accuracy of GCPs
This is a highly error prone process.  To verify all inputs, we have a set of steps. 

- Verify the pixel locations by drawing a circle on the image and cropping.
- Verify the geographic coordinate by cropping a Web-Map-Service frame.
- Writing both images and showing.

In [ ]:
def crop_and_mark_image( image, pix ):

    #  Crop the image
    r = [[pix[0] - 100, pix[0] + 100],[pix[1] - 100, pix[1] + 100]]
    new_img = image[ r[0][0]:r[0][1], r[1][0]:r[1][1] ]

    

In [ ]:
if not os.path.exists( 'crops' ):
    os.system( 'mkdir crops' )

for gcp in gcp_df.itertuples():
    
    pix = np.array( [ gcp.PX, gcp.PY ], dtype = np.float64 )
    lla = np.array( [ gcp.Longitude, gcp.Latitude, gcp.Elevation ], dtype = np.float64 )

    #  Crop scene
    pix_img = crop_and_mark_image( img_bands, pix )
    print( f'Pixel: {pix}, Coord: {lla}' )